In [1]:
# !kaggle datasets download fullmetal26/glovetwitter27b100dtxt
# !unzip glovetwitter27b100dtxt.zip

In [2]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, SimpleRNN, Activation, Dropout, Conv1D
from tensorflow.keras.layers import Embedding, Flatten, LSTM, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import spacy
from sklearn.metrics import classification_report

In [5]:
def load_glove_model(glove_file):
    print("[INFO]Loading GloVe Model...")
    model = {}
    with open(glove_file, 'r', encoding="utf-8") as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embeddings = [float(val) for val in split_line[1:]]
            model[word] = embeddings
    print("[INFO] Done...{} words loaded!".format(len(model)))
    return model
glove_model = load_glove_model("/content/glove.twitter.27B.100d.txt")

[INFO]Loading GloVe Model...
[INFO] Done...531093 words loaded!


In [19]:
# calcultaete number of words
nb_words = len(tokenizer.word_index) + 1

# obtain the word embedding matrix
embedding_matrix = np.zeros((nb_words, 100))
for word, i in word_index.items():
    embedding_vector = glove_model.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Null word embeddings: 10931


In [6]:
# adopted from utils.py
nlp = spacy.load("en_core_web_sm")

def remove_stopwords(sentence):
    '''
    function to remove stopwords
        input: sentence - string of sentence
    '''
    new = []
    # tokenize sentence
    sentence = nlp(sentence)
    for tk in sentence:
        if (tk.is_stop == False) & (tk.pos_ !="PUNCT"):
            new.append(tk.string.strip())
    # convert back to sentence string
    c = " ".join(str(x) for x in new)
    return c


def lemmatize(sentence):
    '''
    function to do lemmatization
        input: sentence - string of sentence
    '''
    sentence = nlp(sentence)
    s = ""
    for w in sentence:
        s +=" "+w.lemma_
    return nlp(s)

def sent_vectorizer(sent, model):
    '''
    sentence vectorizer using the pretrained glove model
    '''
    sent_vector = np.zeros(200)
    num_w = 0
    for w in sent.split():
        try:
            # add up all token vectors to a sent_vector
            sent_vector = np.add(sent_vector, model[str(w)])
            num_w += 1
        except:
            pass
    return sent_vector

In [7]:
# !wget https://raw.githubusercontent.com/haochen23/nlp-rnn-lstm-sentiment/master/training.1600000.processed.noemoticon.csv

In [8]:
# data = pd.read_csv("training.1600000.processed.noemoticon.csv", header=None, encoding='latin-1')
data = pd.read_csv("/content/pre_processed_final.csv")
print(data.shape)
data = data[["sentiment", "text"]]
data = data.dropna()
data.head()

(28777, 3)


,sentiment,text
0,1.0,is lookin 4ward to a long weekend really dont...
1,1.0,myweakness is music and i live to meet the pe...
2,1.0,figured out the internet on my new ipod
3,1.0,cant wait to worship with you guys tonight itl...
4,1.0,congrats james im sure the book is going to b...


In [9]:
data_X = data['text']
data_y = data['sentiment']
data_y = pd.get_dummies(data_y)

In [10]:
data_y = data_y.to_numpy()
data_y

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]], dtype=uint8)

In [11]:
n = 6542
print(data['sentiment'][n], data_y[n])

-1.0 [1 0 0]


In [12]:
max_vocab = 18000
max_len = 15
tokenizer = Tokenizer(num_words=max_vocab)

In [13]:
data_X = data_X.to_numpy()
print(data_X)
tokenizer.fit_on_texts(data_X)
sequences = tokenizer.texts_to_sequences(data_X)
word_index = tokenizer.word_index


['is lookin 4ward to a long weekend  really dont want to go to work 2day tho  x'
 'myweakness  is music and i live to meet the people who make it'
 'figured out the internet on my new ipod' ...
 'speaking of chris jericho it is wednesday today is it not time for some catching up on wrestling'
 'im so jealous of everyone at the justin bieber concert no worries tho ill be seeing maroon 5 in march  roadtrippin'
 'trending tim tebow is now dating cave woman from 10000 bc 1st date question what was it like in a time before jesus  tebowmania']


In [13]:
print('Found %s unique tokens.' % len(word_index))

In [14]:
data_keras = pad_sequences(sequences, maxlen=max_len, padding="post")
# data_keras = data_keras.reshape(data_X.shape[0], 15, 1)
data_keras.shape



(28398, 15)

In [15]:
from sklearn.model_selection import train_test_split
train_X, valid_X, train_y, valid_y = train_test_split(data_keras, data_y, test_size = 0.3, random_state=32)

In [16]:
# adopted from sent_tran_eval.py
def build_model(nb_words, rnn_model="SimpleRNN", embedding_matrix=None):
    '''
    build_model function:
    inputs: 
        rnn_model - which type of RNN layer to use, choose in (SimpleRNN, LSTM, GRU)
        embedding_matrix - whether to use pretrained embeddings or not
    '''
    model = Sequential()
    # add an embedding layer
    if embedding_matrix is not None:
        model.add(Embedding(nb_words, 
                        100, 
                        weights=[embedding_matrix], 
                        input_length= max_len,
                        trainable = False))
    else:
        model.add(Embedding(nb_words, 
                        100, 
                        input_length= max_len,
                        trainable = False))
        
    # add an RNN layer according to rnn_model
    if rnn_model == "SimpleRNN":
        model.add(SimpleRNN(200))
    elif rnn_model == "LSTM":
        model.add(LSTM(200))
    else:
        model.add(GRU(200))
    # model.add(Dense(500,activation='relu'))
    # model.add(Dense(500, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', 
                optimizer='adam',
                metrics=['accuracy'])
    return model

In [23]:
model_rnn = build_model(nb_words, "SimpleRNN", embedding_matrix)
model_rnn.fit(train_X, train_y, epochs=20, batch_size=60,
          validation_data=(valid_X, valid_y), callbacks=EarlyStopping(monitor='val_accuracy', mode='max',patience=3))
predictions = model_rnn.predict(valid_X)
predictions = predictions.argmax(axis=1)
print(classification_report(valid_y.argmax(axis=1), predictions))

Epoch 1/20
332/332 [==============================] - 7s 18ms/step - loss: 0.8800 - accuracy: 0.6044 - val_loss: 0.8185 - val_accuracy: 0.6405
Epoch 2/20
332/332 [==============================] - 5s 15ms/step - loss: 0.7951 - accuracy: 0.6517 - val_loss: 0.7802 - val_accuracy: 0.6585
Epoch 3/20
332/332 [==============================] - 5s 14ms/step - loss: 0.7610 - accuracy: 0.6724 - val_loss: 0.7942 - val_accuracy: 0.6487
Epoch 4/20
332/332 [==============================] - 6s 19ms/step - loss: 0.7336 - accuracy: 0.6867 - val_loss: 0.7926 - val_accuracy: 0.6538
Epoch 5/20
267/267 [==============================] - 2s 5ms/step
              precision    recall  f1-score   support

           0       0.67      0.67      0.67      2801
           1       0.55      0.70      0.62      2633
           2       0.71      0.56      0.63      3086

    accuracy                           0.64      8520
   macro avg       0.64      0.64      0.64      8520
weighted avg       0.65      0.64   

In [21]:
model_lstm = build_model(nb_words, "LSTM", embedding_matrix)
model_lstm.fit(train_X, train_y, epochs=20, batch_size=60,
          validation_data=(valid_X, valid_y), callbacks=EarlyStopping(monitor='val_accuracy', mode='max',patience=3))
predictions = model_lstm.predict(valid_X)
predictions = predictions.argmax(axis=1)
print(classification_report(valid_y.argmax(axis=1), predictions))


Epoch 1/20
332/332 [==============================] - 7s 11ms/step - loss: 0.8123 - accuracy: 0.6360 - val_loss: 0.7443 - val_accuracy: 0.6772
Epoch 2/20
332/332 [==============================] - 2s 7ms/step - loss: 0.7412 - accuracy: 0.6747 - val_loss: 0.7433 - val_accuracy: 0.6749
Epoch 3/20
332/332 [==============================] - 2s 7ms/step - loss: 0.7121 - accuracy: 0.6890 - val_loss: 0.7057 - val_accuracy: 0.6967
Epoch 4/20
332/332 [==============================] - 2s 6ms/step - loss: 0.6828 - accuracy: 0.7062 - val_loss: 0.7008 - val_accuracy: 0.6973
Epoch 5/20
332/332 [==============================] - 2s 7ms/step - loss: 0.6547 - accuracy: 0.7217 - val_loss: 0.6988 - val_accuracy: 0.7005
Epoch 6/20
332/332 [==============================] - 2s 6ms/step - loss: 0.6198 - accuracy: 0.7341 - val_loss: 0.6991 - val_accuracy: 0.6981
Epoch 7/20
332/332 [==============================] - 2s 6ms/step - loss: 0.5843 - accuracy: 0.7532 - val_loss: 0.6966 - val_accuracy: 0.7043
Epoch

In [22]:
model_gru = build_model(nb_words, "GRU", embedding_matrix)
# 20, 120
model_gru.fit(train_X, train_y, epochs=10, batch_size=100,
          validation_data=(valid_X, valid_y), callbacks=EarlyStopping(monitor='val_accuracy', mode='max',patience=3))
predictions = model_gru.predict(valid_X)
predictions = predictions.argmax(axis=1)
print(classification_report(valid_y.argmax(axis=1), predictions))

Epoch 1/10
199/199 [==============================] - 4s 11ms/step - loss: 0.8334 - accuracy: 0.6227 - val_loss: 0.7366 - val_accuracy: 0.6811
Epoch 2/10
199/199 [==============================] - 1s 7ms/step - loss: 0.7455 - accuracy: 0.6778 - val_loss: 0.7243 - val_accuracy: 0.6838
Epoch 3/10
199/199 [==============================] - 1s 6ms/step - loss: 0.7174 - accuracy: 0.6875 - val_loss: 0.7125 - val_accuracy: 0.6907
Epoch 4/10
199/199 [==============================] - 1s 6ms/step - loss: 0.6902 - accuracy: 0.7007 - val_loss: 0.7040 - val_accuracy: 0.6957
Epoch 5/10
199/199 [==============================] - 1s 6ms/step - loss: 0.6698 - accuracy: 0.7125 - val_loss: 0.6975 - val_accuracy: 0.6992
Epoch 6/10
199/199 [==============================] - 1s 6ms/step - loss: 0.6427 - accuracy: 0.7255 - val_loss: 0.7056 - val_accuracy: 0.6914
Epoch 7/10
199/199 [==============================] - 1s 6ms/step - loss: 0.6135 - accuracy: 0.7432 - val_loss: 0.7036 - val_accuracy: 0.6947
Epoch

In [24]:
model_gru.save("rnn.h5")